In [1]:
import os
from bravado.client import SwaggerClient
import pandas as pd
from progressbar import ProgressBar

In [2]:
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
                                    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False})

In [3]:
# read in the genes from the genes.txt file
with open("genes.txt", 'r') as f:
    genes = [gene.strip() for gene in f]
entrezGeneIds = {cbioportal.Genes.getGeneUsingGET(geneId=gene).result().entrezGeneId: gene for gene in genes}

In [4]:
# read in the cancers from the cancers.txt file
with open("cancers.txt", 'r') as f:
    cancers = [cancer.strip() for cancer in f]

In [5]:
for cancer in cancers:
    # if the file exist at queried_data/mutation_matrices/{cancer}_mut_matrix.csv", skip it 
    if os.path.isfile(f"queried_data/mutation_matrices/{cancer}_mut_matrix.csv"):
        print(f"mutation data for {cancer} already exists, skipping")
        continue
    print(f"downloading mutation matrix for {cancer}")

   # get the entire list of sampleIDs in the tcga pan cancer study 
    samples = cbioportal.Samples.getAllSamplesInStudyUsingGET(studyId=f'{cancer}_tcga_pan_can_atlas_2018').response().result
    sample_patient = {sample['sampleId']: sample['patientId'] for sample in samples}

    # create dataframe with sampleIDs as index and patientIDs as columns
    df_blank = pd.DataFrame(index=sample_patient.keys())
    df_patient = pd.DataFrame.from_dict(sample_patient, orient='index', columns=['patient_id'])
    df = pd.merge(df_blank, df_patient, left_index=True, right_index=True)

    muts = cbioportal.Mutations.getMutationsInMolecularProfileBySampleListIdUsingGET(
        molecularProfileId=f'{cancer}_tcga_pan_can_atlas_2018_mutations', # {study_id}_mutations gives default mutations profile for study 
        sampleListId=f'{cancer}_tcga_pan_can_atlas_2018_all', # {study_id}_all includes all samples
        projection="DETAILED" # include gene info
        ).result()
    pbar = ProgressBar()
    for m in pbar(muts):
        # create a column for the gene if it doesn't exist
        if m['gene']['hugoGeneSymbol'] not in df.columns:
            df.loc[:,[m['gene']['hugoGeneSymbol']]] = 0
        df.loc[m['sampleId'],m['gene']['hugoGeneSymbol']] = 1

    # save the dataframe to a csv file
    df.to_csv(f"queried_data/mutation_matrices/{cancer}_mut_matrix.csv")

mutation data for PRAD already exists, skipping
downloading mutation matrix for TGCT


100% |########################################################################|


downloading mutation matrix for UCS


100% |########################################################################|


downloading mutation matrix for MESO


...

100% |########################################################################|


downloading mutation matrix for UCEC


100% |########################################################################|


downloading mutation matrix for GBM


100% |########################################################################|
